## Benchmarks
This demo shows the performance of other SOTA graph embeddings methods and their limitations:
they do not take attributes into account (only can handle discreet attributes)
they are very dependable onto the reinitialization of the random walks and minor graph changes

In [5]:
import os,sys
import random
import numpy as np
import networkx as nx

sys.path.append(os.path.realpath('lib'))
from lib.data_loader import load_local_data
from benchmarks.sub2vec import Sub2vec

### sub2vec demo

In [6]:
dataset_n='aids'
path='data/'
X,y=load_local_data(path,dataset_n, attributes=False, use_node_deg=False)
sub2vec = Sub2vec(property='s', walkLength=100, output='aids_walk', d=128, iter=100, windowSize=2, p=0.5, model='dm')
sub2vec.obtainRandomWalks(X)
embeddings = sub2vec.calculateEmbeddings()
print(embeddings.shape)

Total vects  2000
(2000, 128)


In [ ]:
# re-calculate the embeddings and demonstrate that the cosine similarity doesn't work within rounds 

In [8]:
sub2vec.obtainRandomWalks(X)
embeddings2 = sub2vec.calculateEmbeddings()


Total vects  2000


In [10]:
# display cosine similarity for first 10 embeddings
from sklearn.metrics.pairwise import cosine_similarity
num_graphs, d = embeddings.shape
random_graphs =  np.random.randint(0, num_graphs, size=(10))
for i in random_graphs:
    print(f"Similarity of two graphs within rounds is {cosine_similarity(embeddings[i,:].reshape(1, -1), embeddings2[i,:].reshape(1, -1))}")


Similarity of two graphs within rounds is [[-0.02972129]]
Similarity of two graphs within rounds is [[-0.00695749]]
Similarity of two graphs within rounds is [[0.18097776]]
Similarity of two graphs within rounds is [[-0.0468096]]
Similarity of two graphs within rounds is [[-0.0196318]]
Similarity of two graphs within rounds is [[0.0975268]]
Similarity of two graphs within rounds is [[-0.0731004]]
Similarity of two graphs within rounds is [[0.06960186]]
Similarity of two graphs within rounds is [[0.00319865]]
Similarity of two graphs within rounds is [[-0.16390699]]


In [ ]:
# and now calculate the embeddings in one go and check whether they are similar or not this way

In [ ]:
X_double = np.hstack((X,X))
sub2vec.obtainRandomWalks(X_double)
embeddings1_2 = sub2vec.calculateEmbeddings()


In [21]:
embeddings1 = embeddings1_2[:2000,:]
embeddings2 = embeddings1_2[2000:,:]
num_graphs, d = embeddings1.shape
random_graphs =  np.random.randint(0, num_graphs, size=(10))
for i in random_graphs:
    print(f"Similarity of two graphs from the same round is {cosine_similarity(embeddings1[i,:].reshape(1, -1), embeddings2[i,:].reshape(1, -1))}")
# much better but still very small at times, not consistent and these are the same graphs!

Similarity of two graphs from the same round is [[0.9879497]]
Similarity of two graphs from the same round is [[0.97967386]]
Similarity of two graphs from the same round is [[0.9444059]]
Similarity of two graphs from the same round is [[0.9886148]]
Similarity of two graphs from the same round is [[0.3091755]]
Similarity of two graphs from the same round is [[0.97971815]]
Similarity of two graphs from the same round is [[0.981849]]
Similarity of two graphs from the same round is [[0.5953318]]
Similarity of two graphs from the same round is [[0.9656513]]
Similarity of two graphs from the same round is [[0.98958343]]


In [22]:
# I modified the graphs removing one single (always connected only to one or max two neighbors node)

In [ ]:
from copy import deepcopy
def remove_single_node(graphs_list):
    ''' function just removes a random node from a graph which has one or two neighbors'''
    modified_graphs = []
    for graph in graphs_list:
        nx_graph = graph.nx_graph # get the graph
        degree = nx_graph.degree
        degreeDict = dict(degree)
        random_nodes_to_delete = []
        # pick nodes which have 1 neighbor only
        for node, degree in degreeDict.items():
            if degree == 1:
                random_nodes_to_delete.append(node)
        # else pick nodes which have 2 neighbors
        if len(random_nodes_to_delete) == 0:
            for node, degree in degreeDict.items():
                if degree == 2:
                    random_nodes_to_delete.append(node)
        if len(random_nodes_to_delete)==0:
            modified_graphs.append(graph) # leave the graph as it was
            print('A graph which has no nodes with degrees 1 & 2 detected')
        else:
            copy_graph = deepcopy(graph) # create a new deep copy
            copy_graph.nx_graph.remove_node(random.choice(random_nodes_to_delete))

            modified_graphs.append(copy_graph)
        #
    return modified_graphs


In [ ]:
subset_X = X[:10]
modified_subset = remove_single_node(subset_X)
print('done')
for i in range(10):
    print(f"graph before modif has {len(subset_X[i].nx_graph)} and after modif {len(modified_subset[i].nx_graph)}")
